# Interactive Visualizations with Plotly

https://plot.ly/python/getting-started/

__Plotly currently cannot display plots inside notebooks opened in Jupyter Lab out-of-the-box. Therefore please switch to Jupyter Notebook for this tutorial.__

Jupyter Notebook (if running locally): http://localhost:8888/tree

In [1]:
!conda install plotly --yes

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::matplotlib==3.0.3=py37_1
done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.14
  latest version: 4.7.6

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - plotly


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-2.10                  |         openblas           6 KB  conda-forge
    libblas-3.8.0              |      10_openblas           6 KB  conda-forge
    libcblas-3.8.0             |      10_openblas           6 KB  conda-forge
    liblapack-3.8.0            |      10_openblas           6 KB  conda-forge
    liblapacke-3.8.0           |      10_openblas           6 KB  con

In [16]:
import plotly.offline as plt
import plotly.graph_objs as go

In [8]:
plt.init_notebook_mode(connected=True) # required for plt.iplot
# connected = False requires no internet connection for plotting, 
# but significantly increases notebook size.

## Basic Plotting

In [9]:
plt.plot({
    "data": [go.Scatter(x=[1, 2, 3, 4], y=[4, 3, 2, 1])],
    "layout": go.Layout(title="hello world")
    })

'temp-plot.html'

The *plot* function creates the plot as html file for opening in Browser.

In [10]:
p = plt.iplot({
    "data": [go.Scatter(x=[1, 2, 3, 4], y=[4, 3, 2, 1])],
    "layout": go.Layout(title="hello world")
    })

Classical method to create a plot inside a notebook.

In [15]:
f = go.FigureWidget({
    "data": [go.Scatter(x=[1, 2, 3, 4], y=[4, 3, 2, 1])],
    "layout": go.Layout(title="hello world")
    })
f.show()

New method (Plotly >= 3.0) to create a plot inside a notebook.

## Interactive Plotting of Pandas DataFrames

In [59]:
import pandas as pd
from ipywidgets import widgets

In [60]:
df = pd.read_csv('../code/test_data.csv', parse_dates=[1, 2])
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type
0,2,2018-01-01 00:18:50,2018-01-01 00:24:39,N,1,236,236,5,0.70,6.0,0.5,0.5,0.0,0.0,NaN,0.3,7.3,2,1.0
1,2,2018-01-01 00:30:26,2018-01-01 00:46:42,N,1,43,42,5,3.50,14.5,0.5,0.5,0.0,0.0,NaN,0.3,15.8,2,1.0
2,2,2018-01-01 00:07:25,2018-01-01 00:19:45,N,1,74,152,1,2.14,10.0,0.5,0.5,0.0,0.0,NaN,0.3,11.3,2,1.0
3,2,2018-01-01 00:32:40,2018-01-01 00:33:41,N,1,255,255,1,0.03,-3.0,-0.5,-0.5,0.0,0.0,NaN,-0.3,-4.3,3,1.0
4,2,2018-01-01 00:32:40,2018-01-01 00:33:41,N,1,255,255,1,0.03,3.0,0.5,0.5,0.0,0.0,NaN,0.3,4.3,2,1.0


In [72]:
vendor = widgets.Dropdown(
    options=df.VendorID.unique().tolist(),
    value=1,
    description='Vendor ID:'
    )
passengers = widgets.IntSlider(
    value=1,
    min=df.passenger_count.min(),
    max=df.passenger_count.max(),
    step=1,
    description='Passengers:',
    continuous_update=False,
    )

In [73]:
vendor

Dropdown(description='Vendor ID:', index=1, options=(2, 1), value=1)

In [74]:
passengers

IntSlider(value=1, continuous_update=False, description='Passengers:', max=9)

In [75]:
plot_df = df[(df.VendorID == vendor.value)
             & (df.passenger_count == passengers.value)].head(100)
g = go.FigureWidget(data=[go.Scatter(x=plot_df.trip_distance, y=plot_df.total_amount, 
                                     name='Distance vs. Price', mode='markers')],
                layout=go.Layout(title={'text': 'NYC Taxi Data'},
                                xaxis={'title': 'trip distance'},
                                yaxis={'title': 'total amount'}))

In [76]:
def response(change):
    plot_df = df[(df.VendorID == vendor.value)
                 & (df.passenger_count == passengers.value)].head(100)
    with g.batch_update():
        g.data[0].x = plot_df.trip_distance
        g.data[0].y = plot_df.total_amount

In [77]:
vendor.observe(response, names='value')
passengers.observe(response, names='value')

In [78]:
widgets.VBox([vendor, passengers, g])